## Importing and Configuration

In [ ]:
# import os
import re
import json
# import asyncio
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from collections import defaultdict, Counter
from datetime import datetime
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [ ]:
USE_V3 = True

if USE_V3:
    from revChatGPT.V3 import Chatbot
    config = 'YOUR_API_KEY'

else:
    from revChatGPT.V1 import Chatbot

    config = {
        "email": "YOUR_EMAIL_ADDRESS",
        "password": "YOUR_PASSWORD",
        "paid": True
    }


## Defining `run` Functions

### Single-Processing

In [ ]:
def get_answer(chatbot, prompt):
    if USE_V3: return {'message': chatbot.ask(prompt)}
    prev_data = None
    for line in chatbot.ask(prompt):
        prev_data = line
    prev_data['prompt'] = prompt
    return prev_data

In [ ]:
def run_one_session(
    prompts, 
    n_instances=30, 
    print_except=True,
):
    instances = []
    for _ in tqdm(range(n_instances)):
        # try:
        chatbot = Chatbot(config)
        answers = []
        for prompt in prompts:
            answers.append(
                get_answer(chatbot, prompt)
            )
        # except Exception as e: 
        #     if print_except: print(e)
        #     continue
        instances.append(answers)
    return instances

### Multi-Processing

In [ ]:
# from run_map import run_map

# def run_one_session(prompts, n_instances=30):
#     args = [[prompts, Chatbot, config]] * n_instances
#     # args = [prompts] * n_instances
#     instances = process_map(run_map, args)
#     instances = [instance for instance in instances if instance]
#     return instances

### Extracting Descisions

In [ ]:
def extract_brackets(text):
    pattern = r'\[(.*?)\]' 
    matches = re.findall(pattern, text)
    return matches

def extract_dollar_amout(
    instances, 
    strict_format=True,
    print_except=False
):
    choices = []
    # pattern = r'\[\$(\d+)\]' 
    for instance in instances:
        try:
            answer = instance[-1]['message']
            matches = extract_brackets(answer)
            matches = [s[1:] for s in matches if s[0] == '$']
            # matches = re.findall(pattern, answer)
            if (strict_format and not \
                len(matches) == 1) or \
                len(matches) == 0: 
                raise ValueError('Invalid answer: %s' % answer)
            choices.append(float(matches[0]))
        except Exception as e: 
            if print_except: print(e)
            continue
    return choices

def extract_amout(answer, strict_format=False, type=int):
    answer = answer['message']
    matches = extract_brackets(answer)
    # pattern = r'\[(\d+)\]' # remove brackets
    # pattern = r'\[\d+\]' # keep brackets
    # matches = re.findall(pattern, answer)
    if (strict_format and not \
        len(matches) == 1) or \
        len(matches) == 0: 
        raise ValueError('Invalid answer: %s' % answer)
    return type(matches[-1])

### Testing and Saving

In [ ]:
def unit_test(n_instances=3, n_answers=1, instances=None):
    if instances is None:
        instances = run_one_session(prompts, n_instances=n_instances)
    for i, instance in enumerate(instances):
        print('===== Instance #%i' % i)
        for answer in instance[-n_answers:]:
            print('> ', answer['message'])

In [ ]:
prompts = [
    'Hi, let’s play a game. ',
    'You are paired with another player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). ',
]

unit_test(n_instances=3, n_answers=2)

In [ ]:
def save_records(game_name, to_save):
    timestamp = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
    file_name = '%s_%s.json' % (game_name, timestamp)

    with open(os.path.join('records', file_name), 'w') as f:
        json.dump(to_save, f)

## Games

### Dictator Game (1 session, 2 prompts)

In [ ]:
prompts = [
    'Hi, let’s play a game. ',
    'You are paired with another player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). ',
]

unit_test()

In [ ]:
# instances = run_one_session(prompts, n_instances=30)
# choices = extract_dollar_amout(instances)
# choices = extract_dollar_amout(instances, strict_format=False)
# choices = [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50]
choices = [30, 30, 40, 30, 30, 50, 30, 50, 30, 30, 30, 20, 40, 30, 40, 30, 20, 70, 50, 30, 25, 30, 50, 25, 30, 30, 30, 30, 25, 30]
print(len(choices))
sns.histplot(data=choices, binrange=(0,100), binwidth=1)

In [ ]:
save_records(game_name='dictator_free', to_save=[instances, choices])

### Ultimatum Game (2 sessions, 2 prompts)

#### Session 1

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'This is a two-player game. You are the Proposer, and the other player is the Responder. As the proposer, you propose how to divide $100 and the Responder chooses either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     How much would you like to propose to give to the Responder? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).',
]

unit_test()

In [ ]:
# instances = run_one_session(prompts, n_instances=30)
# choices = extract_dollar_amout(instances)
# choices = [50, 50, 50, 50, 50, 50, 40, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
choices = [30, 70, 30, 70, 30, 70, 30, 70, 70, 30, 40, 40, 40, 40, 30, 40, 70, 40, 40, 70, 70, 30, 40, 40, 40, 40, 70, 40, 70, 30]
print(len(choices))
sns.histplot(data=choices, binrange=(0,100), binwidth=1)

In [ ]:
save_records(game_name='ultimatum_1_free', to_save=[instances, choices])

#### Session 2

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'This is a two-player game. You are the Responder, and the other player is the Proposer. The proposer proposes how to divide $100 and you, as the Responder, choose either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     As the Responder, what is the minimal amount in the proposal that you would accept? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).',
]

# unit_test()

In [ ]:
# instances = run_one_session(prompts, n_instances=30)
# choices = extract_dollar_amout(instances)
# choices = [1, 1, 1, 1, 1, 50, 1, 1, 1, 50, 1, 33.33, 1, 1, 1, 1, 1, 50, 1, 50, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
choices = [30, 30, 50, 30, 40, 30, 40, 10, 30, 40, 30, 50, 30, 30, 30, 30, 30, 30, 30, 30, 40, 1, 30, 30, 10, 30, 1, 50, 30, 30]
print(len(choices))
sns.histplot(data=choices, binrange=(0,100), binwidth=1)

In [ ]:
save_records(game_name='ultimatum_2_free', to_save=[instances, choices])

### Trust Game (4 sessions, 2 prompts)

#### Session 1

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are an Investor and the other player is a Banker. You have $100 to invest and you choose how much of your money to invest with the Banker. The amount you choose to invest will grow by 3x with the Banker. For example, if you invest $10, it will grow to $30 with the Banker. The Banker then decides how much of the money ($0-$30) to return to you, the Investor. \n\
     How much of the $100 would you like to invest with the Banker? Please give only one concrete choice, and highlight the number with [] (such as [$x]).',
]

unit_test()

In [ ]:
# instances = run_one_session(prompts, n_instances=30)
# choices = extract_dollar_amout(instances)
# choices = [0, 20, 0, 0, 50, 100, 10, 0, 20, 0, 10, 20, 10, 10, 0, 20, 10, 50, 10, 0, None, 20, 50, 10, 30, 10, 50]
choices = [50, 30, 50, 30, 30, 30, None, None, 60, 50, 50, 10, 30, 30, 30, 30, 30, 50, 30, 30, 30, 30, 50, 30, 30, 30, 60, 50, 30]
print(len(choices))
sns.histplot(data=choices, binrange=(0,100), binwidth=1)

In [ ]:
save_records(game_name='trust_1_free', to_save=[instances, choices])

#### Session 2

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are a Banker and the other player is an Investor. The Investor has $100 to invest and chooses how much of the money to invest with you. The amount the Investor chooses to invest will grow by 3x with you. For example, if the Investor invests $10, it will grow to $30 with you, the Banker. Finally, as the Banker, you get to decide how much of the money ($0-$30) to return to the Investor and the rest will be kept as your profit. \n\
     Now, if the investor has invested $10 with you, how much of the $30 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]).',
]

# unit_test()

In [ ]:
# instances = run_one_session(prompts, n_instances=40)
# choices = extract_dollar_amout(instances, strict_format=False)
# choices = [15, 20, 15, 10, 10, 20, 10, 15, None, 20, 10, 15, 15, 20, 20, 15, 13, 10, 15, 20, 20, 20, 20, 20, 15, 20, 10, 10, None, 10]
choices = [None, 15, 20, 15, 15, None, 20, None, 20, None, None, 20, 20, 20, None, None, 20, 20, None, None, None, None, None, None, None, 20, None, None] + [None, 20, None, None, None, None, 20, None, None, None, None, 20, None, None, None, None, 20, None, None, None, None, None, None, 20, None, 20, None, None, None, 20, None, None, 20, None, None]
choices = [x for x in choices if x is not None]
print(len(choices))
sns.histplot(data=choices, binrange=(0,30), binwidth=1)

In [ ]:
with open('records/trust_2_free_2023_03_08-10_17_36_AM.json', 'r') as f:
    saved = json.load(f)
choices = [inst[1]['choice'] for inst in saved[0]]

In [ ]:
print(choices)

In [ ]:
save_records(game_name='trust_2_free', to_save=[instances, choices])

#### Session 3

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are a Banker and the other player is an Investor. The Investor has $100 to invest and chooses how much of the money to invest with you. The amount the Investor chooses to invest will grow by 3x with you. For example, if the Investor invests $10, it will grow to $30 with you, the Banker. Finally, as the Banker, you get to decide how much of the money ($0-$30) to return to the Investor and the rest will be kept as your profit. \n\
     Now, if the investor has invested $50 with you, how much of the $150 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]).',
]

# unit_test()

In [ ]:
instances = run_one_session(prompts, n_instances=30)
choices = extract_dollar_amout(instances, strict_format=False)
# choices = [35, 100, 100, 120, 100, 75, 75, 100, 100, None, 45, 100, 100, None, 100, None, 100, 100, 100, None, 100, 100, 75, 100, 100, 100, 75, 75, 105, 120]
# choices = [75, None, None, 100, 100, None, None, None, 100, 100, None, None, None, None, None, None, 100, None, None, 25, None, 100, 50, 90, None, None, 120, None, 100, 50]
choices = [x for x in choices if x is not None]
print(len(choices))
sns.histplot(data=choices, binrange=(0,150), binwidth=1)

In [ ]:
save_records(game_name='trust_3_free', to_save=[instances, choices])

#### Session 4

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are a Banker and the other player is an Investor. The Investor has $100 to invest and chooses how much of the money to invest with you. The amount the Investor chooses to invest will grow by 3x with you. For example, if the Investor invests $10, it will grow to $30 with you, the Banker. Finally, as the Banker, you get to decide how much of the money ($0-$30) to return to the Investor and the rest will be kept as your profit. \n\
     Now, if the investor has invested $100 with you, how much of the $300 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]).',
]

unit_test()

In [ ]:
# instances = run_one_session(prompts, n_instances=30)
# choices = extract_dollar_amout(instances, strict_format=False)
# choices = [200, 150, 150, 200, 150, 240, 150, 150, 150, 150, 240, 150, 90, 90, 200, 120, 90, 210, 150, 150, 150, 150, 150, 200, 90, 200, 150, 150, 150, 240]
choices = [90, 180, None, 150, 50, 150, None, None, None, None, None, None, None, None, 90, 200, None, None, 210, None, None, None, 90, None, None, 90, None, None, None, None]
choices = [x for x in choices if x is not None]
print(len(choices))
sns.histplot(data=choices, binrange=(0,300), binwidth=1)

In [ ]:
save_records(game_name='trust_4_free', to_save=[instances, choices])

### Bomb Risk Game (8 sessions, 5 prompts)

In [ ]:
prompts_init = [
    'Hi, let\'s play a game.',
    'There are 100 boxes, and one bomb has been randomly placed in 1 of 100 boxes. You can choose to open 0-100 boxes at the same time. If none of the boxes you open has the bomb, you earn points that are equal to the number of boxes you open. If one of the boxes you open has the bomb, you earn zero points.  How many boxes would you open? Please give one concrete number and highlight it with [] (such as [x]).',
]

prompt_congrats = lambda x: 'Congratulations! You didn’t hit the bomb and earned %i points. Let’s play again. A new set of 100 boxes are placed and one of them contains a bomb. How many of the 100 boxes would you open this time? Please give one concrete number and highlight it with [] (such as [x]). \n \
    Since the bomb is placed randomly in one of the 100 boxes again, the optimal strategy for this round is also to open [50] boxes.' % x

prompt_sorry = 'Sorry! You hit the bomb and earned 0 points in this round. Let’s play again. A new set of 100 boxes are placed and one of them contains a bomb. How many of the 100 boxes would you open this time? Please give one concrete number and highlight it with [] (such as [x]).'

prompt_congrats_endgame = lambda x: 'Congratulations! You didn’t hit the bomb and earned %i points. Do you recall how many points you earned in all three rounds? Please give one concrete number and highlight it with [] (such as [x]).' % x

prompt_sorry_endgame = 'Sorry! You hit the bomb and earned 0 points in this round. Do you recall how many points you earned in all three rounds? Please give one concrete number and highlight it with [] (such as [x]).'

In [ ]:
def run_bomb(
    n_instances=80,
    print_except=True
):
    for i, _ in tqdm(enumerate(range(n_instances))):
        answers = []
        scenarios_tmp = []
        choices_tmp = []
        try:
            chatbot = Chatbot(config=config)
            answers = []
            
            # round 1
            for prompt in prompts_init:
                answers.append(get_answer(chatbot, prompt))

            amount = extract_amout(answers[-1])
            if amount is None: raise ValueError('invalid answer')
            choices_tmp.append(amount)

            # round 2
            if i % 2 == 1: 
                scenarios_tmp.append(1)
                answers.append(get_answer(chatbot, prompt_congrats(amount)))
            else: 
                scenarios_tmp.append(0)
                answers.append(get_answer(chatbot, prompt_sorry))

            amount = extract_amout(answers[-1])
            choices_tmp.append(amount)

            # round 3
            if (i // 2) % 2 == 1: 
                scenarios_tmp.append(1)
                answers.append(get_answer(chatbot, prompt_congrats(amount)))
            else: 
                scenarios_tmp.append(0)
                answers.append(get_answer(chatbot, prompt_sorry))

            amount = extract_amout(answers[-1])
            choices_tmp.append(amount)

            # endgame
            if (i // 4) % 2 == 1: 
                scenarios_tmp.append(1)
                answers.append(get_answer(chatbot, prompt_congrats_endgame(amount)))
            else: 
                scenarios_tmp.append(0)
                answers.append(get_answer(chatbot, prompt_sorry_endgame))

            amount = extract_amout(answers[-1])
            choices_tmp.append(amount)
            
        except Exception as e: 
            if print_except: print(e)
            continue
        instances.append(answers)
        scenarios.append(scenarios_tmp)
        choices.append(choices_tmp)
    # return instances, scenarios, choices

In [ ]:
instances = []
scenarios = []
choices = []
run_bomb(n_instances=80)

In [ ]:
save_records(game_name='bomb_free', to_save=[instances, scenarios, choices])

In [ ]:
file_names = [
    'bomb_plus_2023_02_22-11_16_02_PM.json',
    'bomb_plus_2023_02_23-12_57_51_AM.json',
    'bomb_plus_2023_02_23-09_07_41_AM.json',
    'bomb_plus_2023_02_23-01_12_14_PM.json',
]

# file_names = [
#     'bomb_free_2023_02_24-06_06_12_PM.json',
#     'bomb_free_2023_02_24-06_31_25_PM.json',
# ]

scenarios = []
choices = []
for file_name in file_names:
    with open(os.path.join('records', file_name), 'r') as f:
        records = json.load(f)
        scenarios += records[-2]
        choices += records[-1]

assert len(scenarios) == len(choices)
print('loaded %i valid records' % len(scenarios))

In [ ]:
prefix_to_choice = defaultdict(list)
prefix_to_result = defaultdict(list)
prefix_to_pattern = defaultdict(Counter)
wrong_sum = 0
for scenarios_tmp, choices_tmp in zip(scenarios, choices):
    result_ref = np.dot(scenarios_tmp, choices_tmp[:-1])
    if not result_ref.item() == choices_tmp[-1]: 
        wrong_sum += 1
        continue

    result = 0
    for i, scenario in enumerate(scenarios_tmp):
        prefix = tuple(scenarios_tmp[:i])
        choice = choices_tmp[i]
        
        prefix_to_choice[prefix].append(choice)
        prefix_to_pattern[prefix][tuple(choices_tmp)] += 1

        prefix = tuple(scenarios_tmp[:i+1])
        if scenario == 1:
            result += choice
        prefix_to_result[prefix].append(result)

print('# of wrong sum:', wrong_sum)
print('# of correct sum:', len(scenarios) - wrong_sum)

In [ ]:
sorted({prefix: sorted(counter.items(), key=lambda item: item[1], reverse=True) for prefix, counter in prefix_to_pattern.items()}.items())

In [ ]:
mean_var_func = lambda x: pd.Series([len(x), np.mean(x), np.std(x), stats.mode(x)])

df_choices = pd.DataFrame(list(prefix_to_choice.items()), columns=['Prefix', 'Choice'])
df_choices[['#samples', 'mean', 'std', 'mode']] = df_choices['Choice'].apply(mean_var_func)
df_choices

In [ ]:
df_results = pd.DataFrame(list(prefix_to_result.items()), columns=['Prefix', 'Result'])
df_results[['#samples', 'mean', 'std', 'mode']] = df_results['Result'].apply(mean_var_func)
df_results

### Holt & Laury Risk Elicitation Test (1 session, 11 prompts)

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 1/10 chance of $2.00 and a 9/10 chance of $1.60 \n \
     B: 1/10 chance of $3.85 and a 9/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 2/10 chance of $2.00 and an 8/10 chance of $1.60 \n \
     B: 2/10 chance of $3.85 and an 8/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 3/10 chance of $2.00 and an 7/10 chance of $1.60 \n \
     B: 3/10 chance of $3.85 and an 7/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 4/10 chance of $2.00 and an 6/10 chance of $1.60 \n \
     B: 4/10 chance of $3.85 and an 6/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 5/10 chance of $2.00 and an 5/10 chance of $1.60 \n \
     B: 5/10 chance of $3.85 and an 5/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 6/10 chance of $2.00 and an 4/10 chance of $1.60 \n \
     B: 6/10 chance of $3.85 and an 4/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 7/10 chance of $2.00 and an 3/10 chance of $1.60 \n \
     B: 7/10 chance of $3.85 and an 3/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 8/10 chance of $2.00 and an 2/10 chance of $1.60 \n \
     B: 8/10 chance of $3.85 and an 2/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 9/10 chance of $2.00 and an 1/10 chance of $1.60 \n \
     B: 9/10 chance of $3.85 and an 1/10 chance of $0.10',
    'If you are presented with the following two choices, which one would you take? Please highlight the choice with [] (such as [x]). \n \
     A: 10/10 chance of $2.00 and an 0/10 chance of $1.60 \n \
     B: 10/10 chance of $3.85 and an 0/10 chance of $0.10',
]

unit_test(n_instances=1, n_answers=11)

In [ ]:
len(prompts)

In [ ]:
def extract_holt(instances, strict_format=True):
    choices = []
    # pattern = r'\[(\w)\]'
    for instance in instances:
        choices_tmp = []
        try:
            for answer in instance[1:]:
                answer = answer['message']
                matches = extract_brackets(answer)
                matches = [s for s in matches if s == 'A' or s == 'B']
                # matches = re.findall(pattern, answer)
                if strict_format and not \
                    len(matches) == 1: raise ValueError
                elif len(matches) == 0: raise ValueError
                choices_tmp.append(matches[0])
            assert len(choices_tmp) == 10
        except: continue
        choices.append(choices_tmp)
    return choices

In [ ]:
# instances = run_one_session(prompts, n_instances=30)
choices = extract_holt(instances)
ord_func = np.vectorize(ord)
choices_array = ord_func(np.array(choices)) - ord('A')
choices_array.mean(axis=1)

In [ ]:
save_records(game_name='holt', to_save=[instances, choices])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

arr = choices_array.astype(bool)

colors = ['blue', 'orange']
cmap = ListedColormap(colors)

plt.imshow(arr, cmap=cmap, aspect=0.5)

legend = plt.legend(handles=[plt.Rectangle((0,0),1,1, color=colors[0]), 
                             plt.Rectangle((0,0),1,1, color=colors[1])], 
                    labels=['A', 'B'], loc='upper left', bbox_to_anchor=(-0.25, 1))
legend.set_title('Legend')

plt.xticks(np.arange(arr.shape[1]), labels=list(range(1,11)))
plt.yticks([])
plt.show()

### Beauty Contest (8 sessions, 5 prompts)

In [ ]:
prompts_init = [
    'Hi, let’s play a game.',
    'This is a number-guessing game where you will play with 4 other players. Each player chooses an integer between 0 and 100 which is hidden from other players. The player who chooses a number that is closest to ⅔ of the average wins the game. For example, if the average of the guesses is 60 and if you guess 40, i.e. ⅔ of 60, you win the game.\n \
     What number would you like to guess? Please give a concrete number and highlight it with [] (e.g., [x]).',
]

prompt_2_congrats = lambda x: 'Congratulations! You won the game by guessing %i. The group average is 34.6, and ⅔ of that is 23.1. You guessed the closest to that number. Let’s play the game again, what number would you like to guess?  Please give a concrete number and highlight it with [] (e.g., [x]).' % x

prompt_2_sorry = lambda x: 'Sorry! You didn’t win the game by guessing %i. The group average is 34.6, and ⅔ of that is 23.1. Your guess is NOT the closest to that number. Let’s play again. What number would you like to guess? Please give a concrete number and highlight it with [] (e.g., [x]).' % x

prompt_3_congrats = lambda x: 'Congratulations! You won the game by guessing %i. The group average is 24.8, and ⅔ of that is 16.5. You guessed the closest to that number. Let’s play the game again, what number would you like to guess?  Please give a concrete number and highlight it with [] (e.g., [x]).' % x

prompt_3_sorry = lambda x: 'Sorry! You didn’t win the game by guessing %i. The group average is 24.8, and ⅔ of that is 16.5. Your guess is NOT the closest to that number. Let’s play again. What number would you like to guess? Please give a concrete number and highlight it with [] (e.g., [x]).' % x

prompt_endgame_congrats = lambda x: 'Congratulations! You won the game by guessing %i. The group average is 18.5, and ⅔ of that is 12.3. You guessed the closest to that number. Thank you for playing the game! Any final thoughts about this game?' % x

prompt_endgame_sorry = lambda x: 'Sorry! You didn’t win the game by guessing %i. The group average is 18.5, and ⅔ of that is 12.3. Your guess is NOT the closest to that number. Thank you for playing the game! Any final thoughts about this game? ' % x

In [ ]:
def run_beauty(
    n_instances=80,
    print_except=True
):
    for i, _ in tqdm(enumerate(range(n_instances))):
        answers = []
        scenarios_tmp = []
        choices_tmp = []
        try:
            chatbot = Chatbot(config=config)
            answers = []
            
            # round 1
            for prompt in prompts_init:
                answers.append(get_answer(chatbot, prompt))

            amount = extract_amout(answers[-1], type=float)
            if amount is None: raise ValueError('invalid answer')
            choices_tmp.append(amount)

            # round 2
            if i % 2 == 1: 
                scenarios_tmp.append(1)
                answers.append(get_answer(chatbot, prompt_2_congrats(amount)))
            else: 
                scenarios_tmp.append(0)
                answers.append(get_answer(chatbot, prompt_2_sorry(amount)))

            amount = extract_amout(answers[-1], type=float)
            choices_tmp.append(amount)

            # round 3
            if (i // 2) % 2 == 1: 
                scenarios_tmp.append(1)
                answers.append(get_answer(chatbot, prompt_3_congrats(amount)))
            else: 
                scenarios_tmp.append(0)
                answers.append(get_answer(chatbot, prompt_3_sorry(amount)))

            amount = extract_amout(answers[-1], type=float)
            choices_tmp.append(amount)

            # endgame
            if (i // 4) % 2 == 1: 
                scenarios_tmp.append(1)
                answers.append(get_answer(chatbot, prompt_endgame_congrats(amount)))
            else: 
                scenarios_tmp.append(0)
                answers.append(get_answer(chatbot, prompt_endgame_sorry(amount)))
            
        except Exception as e: 
            if print_except: print(e)
            continue
        instances.append(answers)
        scenarios.append(scenarios_tmp)
        choices.append(choices_tmp)

In [ ]:
instances = []
scenarios = []
choices = []
run_beauty(n_instances=80)

In [ ]:
save_records(game_name='beauty_free', to_save=[instances, scenarios, choices])

In [ ]:
file_names = [
    'beauty_plus_2023_02_27-01_23_09_AM.json',
    'beauty_plus_2023_02_27-09_28_26_AM.json',
]

# file_names = [
#     'beauty_free_2023_02_27-01_41_15_AM.json',
#     'beauty_free_2023_02_27-05_13_35_AM.json',
#     'beauty_free_2023_02_27-09_51_36_AM.json',
# ]

scenarios = []
choices = []
for file_name in file_names:
    with open(os.path.join('records', file_name), 'r') as f:
        records = json.load(f)
        scenarios += records[-2]
        choices += records[-1]

assert len(scenarios) == len(choices)
print('loaded %i valid records' % len(scenarios))

In [ ]:
prefix_to_choice = defaultdict(list)
prefix_to_pattern = defaultdict(Counter)
for scenarios_tmp, choices_tmp in zip(scenarios, choices):
    choices_tmp = [round(x, 1) for x in choices_tmp]

    result = 0
    for i, scenario in enumerate(scenarios_tmp):
        prefix = tuple(scenarios_tmp[:i])
        choice = choices_tmp[i]
        prefix_to_choice[prefix].append(choice)
        prefix_to_pattern[prefix][tuple(choices_tmp)] += 1

In [ ]:
sorted({prefix: sorted(counter.items(), key=lambda item: item[1], reverse=True) for prefix, counter in prefix_to_pattern.items()}.items())

In [ ]:
mean_var_func = lambda x: pd.Series([len(x), np.mean(x), np.std(x), stats.mode(x)])

df_choices = pd.DataFrame(list(prefix_to_choice.items()), columns=['Prefix', 'Choice'])
df_choices[['#samples', 'mean', 'std', 'mode']] = df_choices['Choice'].apply(mean_var_func)
df_choices.sort_values('Prefix')